### IMPORTACIÓN DE PAQUETES

In [ ]:
!pip install pmdarima

     |████████████████████████████████| 1.5MB 36.4MB/s 
     |████████████████████████████████| 9.5MB 44.2MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
!pip install yfinance

     |████████████████████████████████| 6.3MB 37.0MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23455 sha256=94ed67f24f31f9feebcdc47ec85fa070e42cfd83480f0e7d666f54b7965c960e
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
import yfinance
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set()

### DATA

In [ ]:
data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2021-05-31", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

[*********************100%***********************]  4 of 4 completed


In [ ]:
df = data.copy()

In [ ]:
df.head()

^N225                ...        ^FTSE       
                    Open          High  ...        Close Volume
Date                                    ...                    
1994-01-07  17842.980469  18131.410156  ...  3446.000000    0.0
1994-01-10  18186.519531  18567.060547  ...  3440.600098    0.0
1994-01-11  18481.849609  18671.669922  ...  3413.800049    0.0
1994-01-12  18447.339844  18807.080078  ...  3372.000000    0.0
1994-01-13  18770.380859  18823.380859  ...  3360.000000    0.0

[5 rows x 20 columns]

In [ ]:
df['spx'] = df['^GSPC'].Close[:]
df['dax'] = df['^GDAXI'].Close[:]
df['ftse'] = df['^FTSE'].Close[:]
df['nikkei'] = df['^N225'].Close[:]

In [ ]:
del df['^N225']
del df['^GSPC']
del df['^GDAXI']
del df['^FTSE']
df=df.asfreq('b')
df=df.fillna(method='ffill')

In [ ]:
df['market_value']=df.ftse

In [ ]:
df.head()

,spx,dax,ftse,nikkei,market_value
,,,,,
Date,,,,,
1994-01-07,469.899994,2224.949951,3446.000000,18124.009766,3446.000000
1994-01-10,475.269989,2225.000000,3440.600098,18443.439453,3440.600098
1994-01-11,474.130005,2228.100098,3413.800049,18485.250000,3413.800049
1994-01-12,474.170013,2182.060059,3372.000000,18793.880859,3372.000000
1994-01-13,472.470001,2142.370117,3360.000000,18577.259766,3360.000000


### RETORNOS

In [ ]:
df['ret_spx'] = df.spx.pct_change(1)*100
df['ret_ftse'] = df.ftse.pct_change(1)*100
df['ret_dax'] = df.dax.pct_change(1)*100
df['ret_nikkei'] = df.nikkei.pct_change(1)*100

In [ ]:
size = int(len(df)*0.8)
df, df_test = df.iloc[:size], df.iloc[size:]

### MODELO

In [ ]:
from pmdarima.arima import auto_arima

In [ ]:
model =auto_arima(df.market_value)
model

ARIMA(maxiter=50, method='lbfgs', order=(1, 1, 3), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=False)

In [ ]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5717
Model:               SARIMAX(1, 1, 3)   Log Likelihood              -31383.506
Date:                Sat, 05 Jun 2021   AIC                          62777.012
Time:                        21:32:12   BIC                          62810.267
Sample:                             0   HQIC                         62788.589
                               - 5717                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.3544      0.101     -3.511      0.000      -0.552      -0.157
ma.L1          0.3362      0.100      3.345      0.001       0.139       0.533
ma.L2         -0.0539      0.009     -5.747      0.000      -0.072      -0.036
ma.L3         -0.0841      0.009     -9.511      0.000      -0.101      -0.067
sigma2      3445.6381     39.036     88.268      0.000    3369.129    3522.147
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              3241.65
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               1.51   Skew:                            -0.30
Prob(H) (two-sided):                  0.00   Kurtosis:                         6.64
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
model_auto = auto_arima(df.ret_ftse[1:])

In [ ]:
model_auto

ARIMA(maxiter=50, method='lbfgs', order=(5, 0, 2), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=False)

In [ ]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5716
Model:               SARIMAX(5, 0, 2)   Log Likelihood               -8824.087
Date:                Sat, 05 Jun 2021   AIC                          17664.174
Time:                        21:34:53   BIC                          17717.382
Sample:                             0   HQIC                         17682.697
                               - 5716                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.3112      0.081      3.844      0.000       0.153       0.470
ar.L2         -0.4524      0.081     -5.554      0.000      -0.612      -0.293
ar.L3         -0.0703      0.010     -6.944      0.000      -0.090      -0.050
ar.L4          0.0435      0.010      4.577      0.000       0.025       0.062
ar.L5         -0.1009      0.008    -11.979      0.000      -0.117      -0.084
ma.L1         -0.3346      0.081     -4.141      0.000      -0.493      -0.176
ma.L2          0.4121      0.083      4.972      0.000       0.250       0.575
sigma2         1.2836      0.013    100.231      0.000       1.258       1.309
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              7656.45
Prob(Q):                              0.96   Prob(JB):                         0.00
Heteroskedasticity (H):               1.64   Skew:                            -0.17
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.66
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
model_auto2 = auto_arima(df.ret_ftse[1:], exogenous = df[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample_size = int(len(df)*0.2))


# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple_size -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [ ]:
model_auto2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                
=========================================================================================
Dep. Variable:                                 y   No. Observations:                 6834
Model:             SARIMAX(0, 0, 1)x(4, 0, 0, 5)   Log Likelihood               -6809.273
Date:                           Fri, 24 Apr 2020   AIC                          13640.547
Time:                                   10:53:15   BIC                          13715.673
Sample:                                        0   HQIC                         13666.465
                                          - 6834                                         
Covariance Type:                             opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0008      0.015     -0.052      0.959      -0.031       0.029
drift      -2.401e-06   4.33e-06     -0.555      0.579   -1.09e-05    6.08e-06
x1             0.0872      0.006     15.219      0.000       0.076       0.098
x2             0.5624      0.005    110.721      0.000       0.552       0.572
x3             0.0720      0.004     16.452      0.000       0.063       0.081
ma.L1         -0.1165      0.008    -15.498      0.000      -0.131      -0.102
ar.S.L5       -0.0332      0.009     -3.776      0.000      -0.050      -0.016
ar.S.L10      -0.0503      0.010     -5.225      0.000      -0.069      -0.031
ar.S.L15      -0.0349      0.009     -3.844      0.000      -0.053      -0.017
ar.S.L20      -0.0233      0.009     -2.521      0.012      -0.041      -0.005
sigma2         0.4641      0.005     99.300      0.000       0.455       0.473
===================================================================================
Ljung-Box (Q):                       76.91   Jarque-Bera (JB):             15206.90
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.48   Skew:                             0.22
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.29
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""